**Assignment 2**

---


Christopher Mena -
CAI2300C

In [ ]:
# Step 1 & 2 - Setting Up the Environment

!pip install gradio -qqq

from openai import OpenAI
from scipy.spatial import distance
import numpy as np
import gradio as gr
import pandas as pd
import os     # For github actions to hide my API key


# Ini openAI with my API key
api_key = os.getenv('SECRET_MENA_API_KEY')
client = OpenAI(api_key=api_key)


# Define the embedding function using the OpenAI client
def create_embeddings(texts, model="text-embedding-3-small"):
    embeddings = []
    for text in texts:
        response = client.embeddings.create(
            input=text,
            model=model
        )
        embeddings.append(response.data[0].embedding)
    return embeddings

In [ ]:
# Step 3 - Data Preparation
# Fictional dataset of game reviews for the game, "Kingdom Come Deliverance"

game_reviews = [
    "Absolutely stunning historical accuracy. The combat is tough but rewarding!",
    "A masterpiece of immersion. Feels like I'm really living in medieval Bohemia.",
    "The combat is hard to learn but feels incredibly realistic once mastered.",
    "The attention to detail in this game is insane. Every aspect feels authentic.",
    "One of the best RPGs I've ever played. Deep story and great characters.",
    "Frustrating at times, but that's what makes it feel so realistic!",
    "Not for the casual gamer. The realism is both its greatest strength and weakness.",
    "I love the world-building and the historical setting. Feels like a time machine!",
    "The save system is annoying, but everything else about this game is fantastic.",
    "Hardcore and unforgiving, but if you stick with it, it's an incredible experience.",
    "Finally, an RPG that makes you feel like an actual person in a medieval world!",
    "The game is a bit buggy, but the depth of the world makes up for it.",
    "If you're looking for Skyrim, this ain't it. If you want realism, buy this now.",
    "No magic, no dragons, just pure medieval life. I love it!",
    "The combat system is a steep learning curve, but once you get it, it's amazing.",
    "Realistic to a fault. Sometimes I just want to fast travel without penalties!",
    "The roleplaying aspect is some of the best I've ever experienced in a game.",
    "I was skeptical at first, but this game sucked me in completely.",
    "If you love history and RPGs, you owe it to yourself to play this.",
    "Not perfect, but the most immersive medieval RPG out there.",
    "A buggy mess at times, but I can't stop playing it.",
    "The lack of hand-holding is refreshing. Forces you to think and plan ahead.",
    "The combat is brutal but fair. Every fight feels like a real duel.",
    "The quest writing is fantastic. Every choice has weight.",
    "One of the few RPGs where I actually cared about the protagonist's journey.",
    "The archery mechanics are some of the best I've seen in a game.",
    "The realism makes it frustrating at times, but I keep coming back to it.",
    "Visually stunning, even years after release.",
    "Some mechanics are clunky, but the overall experience is unforgettable.",
    "An underrated gem. More people need to experience this game."
]


In [ ]:
# Embeddings for game reviews
reviews = []
embeddings = create_embeddings(game_reviews, model="text-embedding-3-small")

for review, embedding in zip(game_reviews, embeddings):
    reviews.append({"game review": review, "embedding": embedding})

In [ ]:
# Step 4 - Implementing Semantic Search

# Search query
search_text = "How is the combat?"

# Generate the embedding for the query
search_embedding = create_embeddings([search_text])[0]

# Calculate cosine distances between the query and complaints
distances = []
for review in reviews:
    dist = distance.cosine(search_embedding, review["embedding"])
    distances.append(dist)

# Find the closest complaint
min_dist_ind = np.argmin(distances)
closest_review = reviews[min_dist_ind]

print(f"Search Query: {search_text}")
print(f"Closest Review: {closest_review['game review']}")

Search Query: How is the combat?
Closest Review: The combat is brutal but fair. Every fight feels like a real duel.


In [ ]:
#  Step 5:  Defining the Search function

def find_similar_review(query):
    search_embedding = create_embeddings([query])[0]
    distances = [distance.cosine(search_embedding, c["embedding"]) for c in reviews]
    min_dist_ind = np.argmin(distances)
    closest_review = reviews[min_dist_ind]
    return f"Query: {query}\n\nMost Similar Review: {closest_review['game review']}"

# Create the Gradio interface
interface = gr.Interface(
    fn=find_similar_review,
    inputs="text",
    outputs="text",
    title="Semantic Search for Game Reviews",
    description="Enter a review query to find similar reviews in the database."
)

# Launch the app
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://823b0e9d7d50f909bc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**ADDENDUM** - Enriched Data


---



Using our fictional custom dataset

In [ ]:
# ADDENDUM : Enriched data + 10

enriched_reviews = [
    {"Review": "Absolutely stunning historical accuracy. The combat is tough but rewarding!",
     "Topic": "Historical Accuracy, Combat",
     "Keywords": ["historical accuracy", "combat", "rewarding"]},

    {"Review": "A masterpiece of immersion. Feels like I'm really living in medieval Bohemia.",
     "Topic": "Immersion, World-building",
     "Keywords": ["immersion", "medieval", "Bohemia"]},

    {"Review": "The combat is hard to learn but feels incredibly realistic once mastered.",
     "Topic": "Combat Difficulty, Realism",
     "Keywords": ["combat", "realistic", "difficulty"]},

    {"Review": "The attention to detail in this game is insane. Every aspect feels authentic.",
     "Topic": "Detail, Authenticity",
     "Keywords": ["detail", "authenticity", "medieval"]},

    {"Review": "One of the best RPGs I've ever played. Deep story and great characters.",
     "Topic": "RPG, Story, Characters",
     "Keywords": ["RPG", "story", "characters"]},

    {"Review": "Frustrating at times, but that's what makes it feel so realistic!",
     "Topic": "Frustration, Realism",
     "Keywords": ["frustration", "realistic", "immersion"]},

    {"Review": "Not for the casual gamer. The realism is both its greatest strength and weakness.",
     "Topic": "Realism, Difficulty",
     "Keywords": ["realism", "casual", "difficulty"]},

    {"Review": "I love the world-building and the historical setting. Feels like a time machine!",
     "Topic": "World-building, History",
     "Keywords": ["world-building", "history", "immersion"]},

    {"Review": "The save system is annoying, but everything else about this game is fantastic.",
     "Topic": "Save System, Gameplay",
     "Keywords": ["save system", "annoying", "gameplay"]},

    {"Review": "Hardcore and unforgiving, but if you stick with it, it's an incredible experience.",
     "Topic": "Difficulty, Reward",
     "Keywords": ["hardcore", "unforgiving", "rewarding"]},

    {"Review": "Finally, an RPG that makes you feel like an actual person in a medieval world!",
     "Topic": "Immersion, RPG",
     "Keywords": ["RPG", "immersion", "medieval"]},

    {"Review": "The game is a bit buggy, but the depth of the world makes up for it.",
     "Topic": "Bugs, World Depth",
     "Keywords": ["bugs", "depth", "world"]},

    {"Review": "If you're looking for Skyrim, this ain't it. If you want realism, buy this now.",
     "Topic": "Comparison, Realism",
     "Keywords": ["Skyrim", "realism", "comparison"]},

    {"Review": "No magic, no dragons, just pure medieval life. I love it!",
     "Topic": "Realism, Medieval",
     "Keywords": ["no magic", "realism", "medieval"]},

    {"Review": "The combat system is a steep learning curve, but once you get it, it's amazing.",
     "Topic": "Combat Difficulty, Reward",
     "Keywords": ["combat", "learning curve", "rewarding"]},

    {"Review": "Realistic to a fault. Sometimes I just want to fast travel without penalties!",
     "Topic": "Realism, Fast Travel",
     "Keywords": ["realistic", "fast travel", "penalties"]},

    {"Review": "The roleplaying aspect is some of the best I've ever experienced in a game.",
     "Topic": "Roleplaying, Gameplay",
     "Keywords": ["roleplaying", "gameplay", "elements"]},

    {"Review": "I was skeptical at first, but this game sucked me in completely.",
     "Topic": "Skepticism, Immersion",
     "Keywords": ["skeptical", "immersion", "game"]},

    {"Review": "If you love history and RPGs, you owe it to yourself to play this.",
     "Topic": "History, RPG",
     "Keywords": ["history", "RPG", "must-play"]},

    {"Review": "Not perfect, but the most immersive medieval RPG out there.",
     "Topic": "Imperfections, Immersion",
     "Keywords": ["imperfections", "immersion", "RPG"]},

    {"Review": "A buggy mess at times, but I can't stop playing it.",
     "Topic": "Bugs, Enjoyment",
     "Keywords": ["bugs", "enjoyment", "game"]},

    {"Review": "The lack of hand-holding is refreshing. Forces you to think and plan ahead.",
     "Topic": "Difficulty, Strategy",
     "Keywords": ["hand-holding", "strategy", "depth"]},

    {"Review": "The combat is brutal but fair. Every fight feels like a real duel.",
     "Topic": "Combat, Realism",
     "Keywords": ["combat", "brutal", "duel"]},

    {"Review": "The quest writing is fantastic. Every choice has weight.",
     "Topic": "Writing, Quests",
     "Keywords": ["writing", "quests", "choices"]},

    {"Review": "One of the few RPGs where I actually cared about the protagonist's journey.",
     "Topic": "Protagonist, Story",
     "Keywords": ["protagonist", "journey", "story"]},

    {"Review": "The archery mechanics are some of the best I've seen in a game.",
     "Topic": "Combat, Archery",
     "Keywords": ["archery", "mechanics", "combat"]},

    {"Review": "The realism makes it frustrating at times, but I keep coming back to it.",
     "Topic": "Realism, Frustration",
     "Keywords": ["realism", "frustration", "play"]},

    {"Review": "Visually stunning, even years after release.",
     "Topic": "Graphics, Visuals",
     "Keywords": ["visually stunning", "graphics", "release"]},

    {"Review": "Some mechanics are clunky, but the overall experience is unforgettable.",
     "Topic": "Mechanics, Experience",
     "Keywords": ["clunky", "experience", "gameplay"]},

    {"Review": "An underrated gem. More people need to experience this game.",
     "Topic": "Underrated, RPG",
     "Keywords": ["underrated", "gem", "RPG"]},

    {"Review": "The game world feels alive with NPCs going about their daily routines.",
     "Topic": "World-building, NPCs",
     "Keywords": ["NPCs", "world-building", "dynamic"]},

    {"Review": "I wish there were more customization options for character appearance.",
     "Topic": "Customization",
     "Keywords": ["customization", "appearance", "character"]},

    {"Review": "Combat could use more variety in weapons, but it's still satisfying.",
     "Topic": "Combat, Weapons",
     "Keywords": ["combat", "weapons", "variety"]},

    {"Review": "The soundtrack is incredible. It sets the perfect tone for every scene.",
     "Topic": "Soundtrack, Atmosphere",
     "Keywords": ["soundtrack", "atmosphere", "tone"]},

    {"Review": "Sometimes the AI feels a bit too predictable, but the game is still challenging.",
     "Topic": "AI, Difficulty",
     "Keywords": ["AI", "predictable", "challenge"]},

    {"Review": "The side quests are interesting, but they can feel repetitive at times.",
     "Topic": "Side Quests, Repetition",
     "Keywords": ["side quests", "repetitive", "variety"]},

    {"Review": "The game has a steep learning curve, but once you get into it, it's a lot of fun.",
     "Topic": "Learning Curve, Reward",
     "Keywords": ["learning curve", "rewarding", "fun"]},

    {"Review": "It’s the best RPG I’ve played in years. So immersive and rich with detail.",
     "Topic": "RPG, Immersion",
     "Keywords": ["RPG", "detail", "immersion"]},

    {"Review": "I really appreciate the lack of hand-holding in this game.",
     "Topic": "Difficulty, Challenge",
     "Keywords": ["hand-holding", "difficulty", "challenge"]},

    {"Review": "The pacing can be slow at times, but it’s worth sticking with it.",
     "Topic": "Pacing, Patience",
     "Keywords": ["pacing", "slow", "rewarding"]}
]


In [ ]:
# Embedding creation

for Review in enriched_reviews:
    embedding = create_embeddings([Review["Review"]], model="text-embedding-3-small")[0]
    Review["embedding"] = embedding

In [ ]:
# Search text (game review query)
search_text = "What is the game's difficulty?"
search_embedding = create_embeddings([search_text])[0]

# Calculate cosine distances
distances = []
for Review in enriched_reviews:
    dist = distance.cosine(search_embedding, Review["embedding"])
    distances.append(dist)

# Find the game review with the smallest distance
min_dist_ind = np.argmin(distances)
closest_review = enriched_reviews[min_dist_ind]

print(f"Search query:", search_text)
print(f"Closest Game Review: {closest_review['Review']}")
print(f"Topic: {closest_review['Topic']}")
print(f"Keywords: {', '.join(closest_review['Keywords'])}")

Search query: What is the game's difficulty?
Closest Game Review: The game has a steep learning curve, but once you get into it, it's a lot of fun.
Topic: Learning Curve, Reward
Keywords: learning curve, rewarding, fun


In [ ]:
# Implementing Gradio UI

# Define the Gradio search function
def find_similar_review(query):
    search_embedding = create_embeddings([query])[0]
    distances = [distance.cosine(search_embedding, r["embedding"]) for r in enriched_reviews]
    min_dist_ind = np.argmin(distances)
    closest_review = enriched_reviews[min_dist_ind]
    return (
        f"Query: {query}\n\n"
        f"Closest Game Review: {closest_review['Review']}\n\n"
        f"Topic: {closest_review['Topic']}\n"
        f"Keywords: {', '.join(closest_review['Keywords'])}"
    )

# Create the Gradio interface
interface = gr.Interface(
    fn=find_similar_review,
    inputs="text",
    outputs="text",
    title="Semantic Search for Game Reviews",
    description="Enter a review query to find the most relevant game review."
)

# Launch the app
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bc9a1b83911574edf0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



  For this assignment, I chose a dataset of video game reviews.  Customers can query reviews to answer questions about a particular video game.  In this case, Kingdom Come Deliverance, an open world western RPG similar to Skyrim, Fallout, etc, but in medieval times.  I selected this dataset because I try to play a little bit of video games when I have a little free time, as a way to have some relaxing fun.  Chess, RPGs, simulation, management, turn based tactics.  I thought this would fit nicely for me, while I learn and practice.
	For the methodology, I followed the template provided – creating embeddings, enriching the data, etc.  We calculated cosine distances to identify the closest review based on the minimum distance.  

Example queries :
- #1 - Query: What is the difficulty like?
Closest Game Review: Hardcore and unforgiving, but if you stick with it, it's an incredible experience.<br>
Topic: Difficulty, Reward<br>
Keywords: hardcore, unforgiving, rewarding

- #2 - Query: How is the combat?
Closest Game Review: The combat is brutal but fair. Every fight feels like a real duel.<br>
Topic: Combat, Realism<br>
Keywords: combat, brutal, duel

The semantic search engine using OpenAI seems to work just fine, understanding the context. Enriching the data seems to have helped tremendously between the basic semantic search dataset and the enriched data.

Thank you
